Done:
1. Store root vectors in Phi+ in a list called roots. Store all linear combinations of root vectors with coefficients of 1 in a list called sym_vec_combs
2. Calculate and store the Ualpha under which the possible root vector combinations are not stable.

To do:
1. Make matrices for the root vectors, torus, and unipotent root groups
    1.i Change the data type of the root vectors from numpy arrays to matrices using np.asmatrix https://numpy.org/doc/stable/reference/generated/numpy.asmatrix.html
    
8. Find out if there is a way to tell which orbits lie in a family with one another based on their root vector combination, rather than by calculating the B-orbits by brute force.
2. Calculate all possible B-Orbits
3. For each B-orbit, pull the coefficients on each root vector. Test if each coefficient is zero, and add a list of all coefficients of 0 to a list that stores the root vectors in the zero set. Do the same for coefficients that have only entries from T and none from the Ualpha
4. __Automate the discovery of algebraic relationships between the coordinates to give the polynomial defining equations in the zero set and the nonzero set, probably using an algorithm in elimination theory. "The basic idea of the implicitization problem is to convert the parametrization into definining equations for V" (Cox Little O'Shea pp. 133. N.t. they use V to refer to the polynomial zero set)__
5. __Find a way to tell when an "orbit" is infinite. It's probably when there's an algebraic dependence relation among the coordinates that involves a coefficient on one of the root vectors (in addition to or instead of the variables standing for an arbitrary scalar entry in T or Ualpha). Find a rigorous way of stating and defending this. Modality is probably the useful concept here. Since the orbits are varieties, is it possible to calculate the modality of the individual "orbits" to see if they turn out to be families of orbits? Then an orbit family would have modality 1, and a singleton orbit would have modality zero.__
6. __Automate the reverse inclusion calculations in Python. Is it possible that it actually isn't necessary to check the reverse inclusion? When would it happen that the B-orbit is in the intersection of the zero set and the nonzero set but not vice versa?__
7. __Find a way to tell when we have calculated all the B-orbits. Automate proof of nilradical exhaustion. Is there a way we can tell how many infinite "orbits" and how many singleton "orbits" there are before we calculate them? If so, we could just keep count and do a proof of exhaustion of n when we've got enough.__
9. Put in the option to display a B-orbit in equation form
10. Add error handling

In [4]:
import numpy as np
from itertools import combinations
from collections import OrderedDict

In [5]:
# Take the type and rank of the Lie group
L = RootSystem(["B",3]).root_space()
roots=list(L.positive_roots())
print(roots)
#roots[0].to_vector()
roots = [root.to_vector() for root in roots]
roots

[alpha[1], alpha[2], alpha[3], alpha[1] + alpha[2], alpha[2] + 2*alpha[3], alpha[2] + alpha[3], alpha[1] + alpha[2] + 2*alpha[3], alpha[1] + alpha[2] + alpha[3], alpha[1] + 2*alpha[2] + 2*alpha[3]]


[(1, 0, 0),
 (0, 1, 0),
 (0, 0, 1),
 (1, 1, 0),
 (0, 1, 2),
 (0, 1, 1),
 (1, 1, 2),
 (1, 1, 1),
 (1, 2, 2)]

In [6]:
"""
Create a list of all combinations we want to sum of root vectors as symbols, 
excluding the empty sum but including single root vectors
"""
x_list=roots
sym_vec_combs = []
for i in range(1, len(x_list) + 1):
    sym_vec_combs += list(combinations(x_list,i))
print(len(sym_vec_combs))

511


In [7]:
"""
Fix a root x in Phi+, then loop through each root xi in Phi+ to check
whether x+xi is in Phi+. If x+xi is in Phi+, then x is not stable 
under Ui and we place Ui in a list with the same indexing as roots.

Index zero corresponds to alpha1, etc.
"""
Ualpha_list=[ [] for _ in range(len(roots)) ]
N = range(len(roots))
for f in N:
    for t in N:
        if roots[f]+roots[t] in roots:
            Ualpha_list[f].append(roots[t])

print(Ualpha_list)

[[(0, 1, 0), (0, 1, 2), (0, 1, 1)], [(1, 0, 0), (0, 0, 1), (1, 1, 2)], [(0, 1, 0), (1, 1, 0), (0, 1, 1), (1, 1, 1)], [(0, 0, 1), (0, 1, 2)], [(1, 0, 0), (1, 1, 0)], [(1, 0, 0), (0, 0, 1), (1, 1, 1)], [(0, 1, 0)], [(0, 0, 1), (0, 1, 1)], []]


In [8]:
"""
Create a list xUlist to store the Ualpha under which the possible 
root vector combinations (stored in sym_vec_combs) are not stable.
Shares index with sym_vec_combs.
"""
xUlist = [ [] for _ in range(len(sym_vec_combs)) ]
for k in range(len(sym_vec_combs)):
    for i in range(len(Ualpha_list)):
        if roots[i] in sym_vec_combs[k]:
            xUlist[k].extend(Ualpha_list[i])

In [9]:
"""
Make matrices for the root vectors in type A
"""
# sl(l+1,C)
l = 2
"""
For i greater than or equal to one and less than or equal to l, append a matrix with a 1 in the ith row and jth column 
and a -1 in the l+j row and the l+i column to a list called mat_list
"""


'\nFor i greater than or equal to one and less than or equal to l, append a matrix with a 1 in the ith row and jth column \nand a -1 in the l+j row and the l+i column to a list called mat_list\n'

In [149]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays
"""
def make_rvecs_B(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_ij for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        rvecs.setdefault(hilo,M)
        M = np.zeros((2*l+1,2*l+1),dtype = int)
        #print(i)
        
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1),dtype = int)
    #print(i,j)
    return rvecs

rvecs = make_rvecs_B(3)
print(len(rvecs))

9


In [11]:
"""
Make matrices for the root vectors for the positive roots in type C.
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays
"""
def make_rvecs_C(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1))
    # pii
    for i in np.arange(1,l+1,1):
        M[i,l+i]=1
        hilo = "p" + str(i) + str(i)
        rvecs.setdefault(hilo,M)
        M = np.zeros((2*l+1,2*l+1))
    
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1))
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = 1
                hilo = "p" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1))
    #print(i,j)
    return rvecs

rvecs = make_rvecs_C(2)
print(len(rvecs))

4


In [112]:
"""
Make matrices for the root vectors for the positive roots in type D with l\geq 2, so that L is semisimple.
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 133
of Erdmann and the values are the eigenvectors as numpy arrays
"""
def make_rvecs_D(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1))
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1))
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1))
    #print(i,j)
    return rvecs

rvecs = make_rvecs_D(3)
print(rvecs)

{'b1': array([[ 0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 'b2': array([[ 0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 'b3': array([[ 0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 'p12': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],


<>:5: DeprecationWarning: invalid escape sequence \g
<>:5: DeprecationWarning: invalid escape sequence \g
<>:5: DeprecationWarning: invalid escape sequence \g
<ipython-input-112-e1bb5fe06171>:5: DeprecationWarning: invalid escape sequence \g
  """


In [95]:
"""
Make a matrix for the torus in type B
"""
# Make a matrix for the torus given group type and rank
# Input for Gtype may be A,B,C,D
# None of these are accurate except the one for type B
def make_T(Gtype,l):
    if Gtype == 'A':
        n = l + 1
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    elif Gtype == 'B':
        n = 2*l+1 
        syms = np.array([var("q"+str(i)) for i in range(l)])
        symsinv = np.array([var("q"+str(i))^(-1) for i in range(l)])
        syms = np.insert(syms,0,1)
        syms = np.insert(syms,l+1,symsinv)
        #print(syms)
        T = np.diag(syms)
    elif Gtype == 'C':
        n = 2*l
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    elif Gtype == 'D':
        n = 2*l
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    return T

In [98]:
make_T('B',4)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, q0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, q1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, q2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, q3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1/q0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1/q1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1/q2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1/q3]], dtype=object)

In [123]:
import sympy
from sympy import Symbol, Matrix

In [142]:
mat = sympy.Matrix(rtry)
t = Symbol('t')
(t*mat).exp()

Matrix([
[1.0, 0,     0,   0,      0, 0,   0],
[  0, 1, 1.0*t,   0,      0, 0,   0],
[  0, 0,   1.0,   0,      0, 0,   0],
[  0, 0,     0, 1.0,      0, 0,   0],
[  0, 0,     0,   0,    1.0, 0,   0],
[  0, 0,     0,   0, -1.0*t, 1,   0],
[  0, 0,     0,   0,      0, 0, 1.0]])

In [150]:
rtry = rvecs["m12"]
rtry

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [155]:
Ua = {}
t = var('t')
for key, value in rvecs.items():
    x = value*t
    print(x)
    Ua.setdefault(key,e^x)
    
Ua

[[0 0 0 0 -t 0 0]
 [t 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


TypeError: unable to convert array([[0, 0, 0, 0, -t, 0, 0],
       [t, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object) to a symbolic expression

In [99]:
def make_U(Gtype,l):
    Ua = {}

    M = np.eye((2*l+1,2*l+1))
    # Add root vectors b_ij for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        cord = "bt" + str(i)
        M[i,0] = var(cord)
        M[0,l+i] = -var(cord)
        hilo = "b" + str(i)
        rvecs.setdefault(hilo,M)
        M = np.eye((2*l+1,2*l+1))
        #print(i)
        
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.eye((2*l+1,2*l+1))
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.eye((2*l+1,2*l+1))
    #print(i,j)
    return rvecs

In [38]:
I = np.asmatrix(np.eye(2))
a = var('a')
b = var('b')
listy = np.array([a,b])
x = listy*I
x

matrix([[1.0*a, 1.0*b]], dtype=object)